In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from Training import GetDatasets
from Nets  import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from Simulation import buffer_simulation, save_obj, load_obj


tf.random.set_seed(1234)
np.random.seed(1234)

In [2]:
import tensorflow as tf
import numpy as np
a = np.array([1, 2, 3])
b = tf.constant(a)
a[0] = 4
print(b)  # tf.Tensor([4 2 3], shape=(3,), dtype=int64)


tf.Tensor([4 2 3], shape=(3,), dtype=int32)


a) Get Dataset

In [8]:
train_batch_size = test_batch_size = 32

train_set,valid_set,test_set = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, train_batch_size, test_batch_size)

a) Get Model

In [9]:
import tensorflow_datasets as tfds
tfds.load('colorectal_histology')

{'train': <PrefetchDataset shapes: {filename: (), image: (150, 150, 3), label: ()}, types: {filename: tf.string, image: tf.uint8, label: tf.int64}>}

Se crea la red, sin activar la cuantización ni el efecto de envejecimiento
Las dimensiones de entrada de la imagen (227,227), el número de clases (8) y el tamaño de los batches

In [10]:
activation_aging = [True] * 21
VGG16   = GetNeuralNetworkModel('VGG16',(227,227,3),8, quantization = False, aging_active=activation_aging)
loss      = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
VGG16.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [6]:
VGG16.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
lambda_39 (Lambda)           (None, 227, 227, 3)       0         
_________________________________________________________________
lambda_40 (Lambda)           (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 227, 227, 64)      1792      
_________________________________________________________________
lambda_41 (Lambda)           (None, 227, 227, 64)      0         
_________________________________________________________________
re_lu_15 (ReLU)              (None, 227, 227, 64)      0         
_________________________________________________________________
lambda_42 (Lambda)           (None, 227, 227, 64)      0   

In [7]:
for index,layer in enumerate(VGG16.layers):
    print(index,layer.name)

0 input_2
1 lambda_39
2 lambda_40
3 conv2d_13
4 lambda_41
5 re_lu_15
6 lambda_42
7 conv2d_14
8 lambda_43
9 re_lu_16
10 lambda_44
11 max_pooling2d_5
12 lambda_45
13 conv2d_15
14 lambda_46
15 re_lu_17
16 lambda_47
17 conv2d_16
18 lambda_48
19 re_lu_18
20 lambda_49
21 max_pooling2d_6
22 lambda_50
23 conv2d_17
24 lambda_51
25 re_lu_19
26 lambda_52
27 conv2d_18
28 lambda_53
29 re_lu_20
30 lambda_54
31 conv2d_19
32 lambda_55
33 re_lu_21
34 lambda_56
35 max_pooling2d_7
36 lambda_57
37 conv2d_20
38 lambda_58
39 re_lu_22
40 lambda_59
41 conv2d_21
42 lambda_60
43 re_lu_23
44 lambda_61
45 conv2d_22
46 lambda_62
47 re_lu_24
48 lambda_63
49 max_pooling2d_8
50 lambda_64
51 conv2d_23
52 lambda_65
53 re_lu_25
54 lambda_66
55 conv2d_24
56 lambda_67
57 re_lu_26
58 lambda_68
59 conv2d_25
60 lambda_69
61 re_lu_27
62 lambda_70
63 max_pooling2d_9
64 lambda_71
65 flatten_1
66 dense_3
67 lambda_72
68 re_lu_28
69 lambda_73
70 dense_4
71 lambda_74
72 re_lu_29
73 lambda_75
74 dense_5
75 lambda_76
76 tf.compat.v1

 b) Load/Save Weigths

In [11]:
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'VGG16')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir, 'Weights') 
VGG16.load_weights(wgt_dir)

In [14]:
for layer in VGG16.layers:
    weights = layer.get_weights()
    if weights:
        print('capa',layer.name)

capa conv2d_13
capa conv2d_14
capa conv2d_15
capa conv2d_16
capa conv2d_17
capa conv2d_18
capa conv2d_19
capa conv2d_20
capa conv2d_21
capa conv2d_22
capa conv2d_23
capa conv2d_24
capa conv2d_25
capa dense_3
capa dense_4
capa dense_5


Evaluación del accuracy y loss de la red

In [7]:
(OrigLoss,OrigAcc) = VGG16.evaluate(test_set)
print(test_set)

24/24 [==============================] - 126s 5s/step - loss: 0.5269 - accuracy: 0.7920
<BatchDataset shapes: ((None, 227, 227, 3), (None, 8)), types: (tf.float32, tf.float32)>


2) Stats

 Write/Read Stats¶

Se identifican (manualmente) las capas procesadadas(Convoluciones, Full conectadas y Pooling) junto a las capas que contienen los resultados que se escribiran en el buffer (capas luego de la funcion de activacion y/o Normalizacion)

In [ ]:
for index,layer in enumerate(VGG16.layers):
    print(index,layer.name)


Con el siguiente bloque obtenemos el número de lecturas y escrituras por posición de memoria tanto usando la estrategia de CNN Gating o sin usarla

In [ ]:
num_address  = 290400
Indices      = [0,3,9,11,17,19,25,31,37,40,45,50]
samples      = 2
# Sin Power Gating:
Data         = GetReadAndWrites(VGG16,Indices,num_address,samples,CNN_gating=False)
stats        = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
Baseline_Acceses   = pd.DataFrame(stats).reset_index(drop=False)
# Con Power Gating
Data     = GetReadAndWrites(VGG16,Indices,num_address,samples,CNN_gating=True)
stats    = {'Lecturas': Data['Reads'],'Escrituras': Data['Writes']}
CNN_gating_Acceses = pd.DataFrame(stats).reset_index(drop=False)
#save_obj(Data,'Data/Acceses/VGG16/Colorectal_Dataset/Baseline')
#save_obj(stats,'Data/Acceses/VGG16/Colorectal_Dataset/CNN_gating_Adj')

Analizar la posibilidad de usar menos bits

In [ ]:
CheckAccuracyAndLoss('VGG16', test_set, wgt_dir, act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                    input_shape = (227,227,3), output_shape = 8, batch_size = test_batch_size);

 c) Activation Stats

Para la configuración anterior, se observa el valor medio,maximo,minimo y el ratio de saturación tanto de las activaciones procesadas dentro de la unidad matricial de multiplicacion como de las almacenadas en el buffer. Nota: el ultimo parametro indica el numero de iteraciones que se deben realizar hasta agotar el dataset, se obtiene como numero de imagenes del dataset dividido en el batch size.

In [ ]:
ActivationStats(VGG16,test_set,11,4,24)

3) Buffer Simulation

 a) Baseline

Ahora para el Baseline simularemos el comportamiento de 1 buffer durante la inferencia de 3 imagenes (solo 3 como ejemplo), la red se crea ahora activando la cuantizacion pero no el envejecimiento. LI y AI son los definidos en el item 2) Stats

......estos ficheros ya están , solo cargar e interpretarlos

Sin inyectar errores original

In [ ]:
# from copy import deepcopy
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from datetime import datetime
import itertools
from Training import GetDatasets
import numpy as np
import os

cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'VGG16')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')
    

trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)


Accs_x= []



network_size   = 290400*16   # Tamaño del buffer (en bits)
num_of_samples = 1       # Numero de muestras de distintas configuraciones de fallos a testear por cada valor de Accs/Loss
    
n_bits_total = np.ceil(network_size).astype(int)    #numero de bits totales
    
buffer = np.array(['x']*(network_size))
print(buffer)
for index in range(0,num_of_samples):
        
    loss,acc_x   = CheckAccuracyAndLoss('VGG16', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = False)
       
        
    print(acc_x)
        
      
        
    Accs_x.append(acc_x)
        
print(str(n_bits_total)+' completada: ', datetime.now().strftime("%H:%M:%S"))
#save_obj(Accs_x,'Data/Errors/VGG16/Colorectal Dataset/Accs_x')
#save_obj(Loss,'Data/Errors/VGG16/Colorectal Dataset/Loss')

# Analizando el fichero en su estado original

In [ ]:
## from copy import deepcopy
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
import collections
from datetime import datetime
import itertools
from Training import GetDatasets
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import os, sys

 
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'VGG16')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')
    

trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)


Accs = []
Accs_w = []
Accs_a_w = []



buffer_size= 16777216

       
ruta_bin = 'Data\Fault Characterization\VC707\RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
directorio = pathlib.Path(ruta_bin)
num_of_samples = 1

variantes= [analize_file, analize_file_uno, analize_file_uno_ceros]
ficheros = [fichero.name for fichero in directorio.iterdir()]
for index in range(0,num_of_samples):
    for i, j in enumerate(ficheros):
        directorio= os.path.join(ruta_bin, j)
        buffer= (analize_file(directorio, buffer_size))
        error_mask, locs = (buffer_vectores(buffer))
    
        loss,acc   = CheckAccuracyAndLoss('VGG16', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
        loss,acc_w   = CheckAccuracyAndLoss('VGG16', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
        loss,acc_a_w   = CheckAccuracyAndLoss('VGG16', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                            act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                            batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = True,
                                            faulty_addresses = locs, masked_faults = error_mask)
    
    
        Accs.append(acc)
        Accs_w.append(acc_w)
        Accs_a_w.append(acc_a_w)                                      


Acc=pd.DataFrame(Accs)
Acc_w =pd.DataFrame(Accs_w)
Acc_a_w =pd.DataFrame(Accs_w)                                         
#buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
#buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='ceros_707', index=False)


Acc.head()
    
   



    
        
    

print(str()+' operación completada: ', datetime.now().strftime("%H:%M:%S"))
#lsave_file(Accs,'Data/Fault Characterization/Accs')

#save_obj(Loss,'Data/Errors/VGG16/Colorectal Dataset/Loss')

# Inyectando errores en 1

In [ ]:
from Stats import CheckAccuracyAndLoss
from Simulation import save_obj, load_obj
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
import collections
from datetime import datetime
import itertools
from Training import GetDatasets
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
import os, sys

trainBatchSize = testBatchSize = 16
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)




    

cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'VGG16')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')
    

Accs_1 = []
Accs_w_1 = []
Accs_a_w_1 = []



df = pd.DataFrame(columns=['Accs', 'Acc_w'])
buffer_size= 16777216
num_of_samples = 1


       
ruta_bin = 'Data\Fault Characterization\VC707\RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
directorio = pathlib.Path(ruta_bin)

variantes= [analize_file, analize_file_uno, analize_file_uno_ceros]
ficheros = [fichero.name for fichero in directorio.iterdir()]
print(directorio)
for index in range(0,num_of_samples):
    for i, j in enumerate(ficheros):
        directorio= os.path.join(ruta_bin, j)
        buffer= (analize_file_uno(directorio, buffer_size))
        error_mask, locs = (buffer_vectores(buffer))
        print(directorio)
        
        loss,acc_1   = CheckAccuracyAndLoss('VGG16', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_w_1   = CheckAccuracyAndLoss('VGG16', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_a_w_1   = CheckAccuracyAndLoss('VGG16', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        
        Accs_1.append(acc_1)
        Accs_w_1.append(acc_w_1)
        Accs_a_w_1.append(acc_a_w_1)                                      


Acc_1=pd.DataFrame(Accs_1)
Acc_w_1 =pd.DataFrame(Accs_w_1)
Acc_a_w_1 =pd.DataFrame(Accs_w_1)                                         
#buf_cero = pd.concat([Acc_1,Acc_w_1, Acc_a_w_1], axis=1, join='outer')
#buf_cero.columns =['Acc_uno', 'A_w_uno', 'Acc_a_w_uno'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='unos_707', index=False)

#buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
#buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='ceros_707', index=False)



Acc.head()
    
   



    
        
    

print(str()+' operación completada: ', datetime.now().strftime("%H:%M:%S"))
save_file(Accs_1,'Data/Fault Characterization/Accs')

#save_obj(Loss,'Data/Errors/VGG16/Colorectal Dataset/Loss')

# Inyectando errores aleatorios ceros y unos

In [ ]:
buffer_size= 16777216
Accs_1_0 = []
Accs_w_1_0 = []
Accs_a_w_1_0 = []



df = pd.DataFrame(columns=['Accs', 'Acc_w'])
buffer_size= 16777216
num_of_samples = 1


       
ruta_bin = 'Data\Fault Characterization\VC707\RawData'
#ruta_bin = 'Data/Fault Characterization/KC705_B/RowData/'
directorio = pathlib.Path(ruta_bin)

variantes= [analize_file, analize_file_uno, analize_file_uno_ceros]
ficheros = [fichero.name for fichero in directorio.iterdir()]

for index in range(0,num_of_samples):

    for i, j in enumerate(ficheros):
        directorio= os.path.join(ruta_bin, j)
        buffer= (analize_file_uno_ceros(directorio, buffer_size))
        error_mask, locs = (buffer_vectores(buffer))
        print(directorio)
        
        loss,acc_1_0   = CheckAccuracyAndLoss('VGG16', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = False,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_w_1_0   = CheckAccuracyAndLoss('VGG16', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = False, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        loss,acc_a_w_1_0   = CheckAccuracyAndLoss('VGG16', test_dataset, wgt_dir, output_shape=8, input_shape = (227,227,3),
                                                act_frac_size = 11, act_int_size = 4, wgt_frac_size = 11, wgt_int_size = 4,
                                                batch_size=testBatchSize, verbose = 0, aging_active = True, weights_faults = True,
                                                faulty_addresses = locs, masked_faults = error_mask)
        
        
        Accs_1_0.append(acc_1_0)
        Accs_w_1_0.append(acc_w_1_0)
        Accs_a_w_1_0.append(acc_a_w_1_0)                                      


Acc_1_0=pd.DataFrame(Accs_1_0)
Acc_w_1_0 =pd.DataFrame(Accs_w_1_0)
Acc_a_w_1_0 =pd.DataFrame(Accs_w_1_0)                                         
buf_cero = pd.concat([Acc,Acc_w, Acc_a_w,Acc_1,Acc_w_1, Acc_a_w_1,Acc_1_0,Acc_w_1_0, Acc_a_w_1_0], axis=1, join='outer')
buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero', 'Acc_uno', 'A_w_uno', 'Acc_a_w_uno' ,'Acc_uno_cero', 'A_w_uno_cero', 'Acc_a_w_uno_cero'] 
buf_cero.to_excel('resultado.xlsx', sheet_name='fichero_707', index=False)

#buf_cero = pd.concat([Acc_1,Acc_w_1, Acc_a_w_1], axis=1, join='outer')
#buf_cero.columns =['Acc_uno', 'A_w_uno', 'Acc_a_w_uno'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='unos_707', index=False)

#buf_cero = pd.concat([Acc,Acc_w, Acc_a_w], axis=1, join='outer')
#buf_cero.columns =['Acc_cero', 'A_w_cero', 'Acc_a_w_cero'] 
#buf_cero.to_excel('resultado.xlsx', sheet_name='ceros_707', index=False)
Acc.head()
    
   



    
        
    

print(str()+' operación completada: ', datetime.now().strftime("%H:%M:%S"))
#save_file(Accs_1,'Data/Fault Characterization/Accs')

#save_obj(Loss,'Data/Errors/VGG16/Colorectal Dataset/Loss')

# Gráficas